In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08789205551147461
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 31.65357497531754
200000 51.64680893972166
300000 60.45176606689934
400000 57.60252839215908
500000 56.0189983749206
600000 57.54256738547695
700000 59.2035327940703
800000 59.784893830563455
900000 69.92348568007021
1000000 80.76535536463537
1100000 79.62809249982017
1200000 82.81239587060236
1300000 29.420446960429956
1400000 88.67220974860882
1500000 98.63874027608692
1600000 96.78341801576173
1700000 96.99763585452575
1800000 97.52953702086722
1900000 103.75289395061442
2000000 109.63713888502195
2100000 103.17084641427223
2200000 111.0129138796046
2300000 105.07415921117803
2400000 97.20649078182191
2500000 87.35273244189301
2600000 32.919199584982
2700000 102.73411823036713
2800000 103.26587918031424
2900000 98.60617465986697
3000000 98.90428821934

29400000 108.00243065506014
29500000 103.26476539036794
29600000 105.29403192006923
29700000 109.58366566326922
29800000 107.19116758369687
29900000 109.97284427889039
30000000 66.41454689333587
30100000 86.56568306343455
30200000 114.5607251405931
30300000 109.90039953536686
30400000 86.3636884330516
30500000 58.47260896808891
30600000 107.83447143915568
30700000 108.32716100089395
30800000 108.53047206736203
30900000 105.81716602931114
31000000 110.21669999029702
31100000 106.66734570401363
31200000 104.48366920519892
31300000 104.26232542273306
31400000 100.90452696380152
31500000 19.45952831485225
31600000 111.49485247624862
31700000 103.89182646790968
31800000 100.65091196655568
31900000 106.97799074861467
32000000 105.22204427824184
32100000 106.73046561832412
32200000 107.5247484573633
32300000 106.60856901568562
32400000 106.47739588894207
32500000 61.59042035167852
32600000 87.43887339393594
32700000 109.48956784697114
32800000 113.65448273811622
32900000 115.73337728592165
33

59200000 44.9750226121257
59300000 96.90095831091998
59400000 97.36354164847253
59500000 99.0507597407505
59600000 103.10222526045095
59700000 101.36287571044028
59800000 102.63371079445153
59900000 102.99817378083823
60000000 38.17714013751862
60100000 112.68166617495055
60200000 106.2589375660895
60300000 109.6650325602494
60400000 106.00111418248103
60500000 101.53103928462986
60600000 99.21968189692991
60700000 103.29367154050533
60800000 93.88908350674615
60900000 49.18361629835617
61000000 101.28203160305551
61100000 96.05404435633255
61200000 97.20704718900487
61300000 96.69170932767842
61400000 90.88003541304974
61500000 95.63814414901805
61600000 87.08852198392216
61700000 28.665522701866543
61800000 93.08173202567205
61900000 90.58733698160013
62000000 91.9679866898461
62100000 90.90596270164608
62200000 92.06747486622784
62300000 100.48249240513118
62400000 100.4941755035785
62500000 35.79562673527006
62600000 106.50375024734129
62700000 108.31634133456024
62800000 99.939450

89400000 53.5366619577954
89500000 89.85149498041861
89600000 93.39743341342194
89700000 91.32564022058817
89800000 96.1000068160135
89900000 74.18232179071029
90000000 59.76430222155453
90100000 93.38105979870139
90200000 95.59468870280003
90300000 98.57441983451102
90400000 96.03228545953122
90500000 108.49180586727365
90600000 94.19419114410823
90700000 101.96480669834955
90800000 94.8225271780868
90900000 97.01996478808076
91000000 92.72185859155125
91100000 91.91945877653815
91200000 83.46530466919205
91300000 38.00988198112744
91400000 89.40768491295673
91500000 90.34071664392253
91600000 91.01205157968123
91700000 85.75711776489034
91800000 78.63328275600118
91900000 39.2368522903445
92000000 92.20413905032358
92100000 87.12846349871452
92200000 89.81256595018932
92300000 88.83548374754692
92400000 72.3281879907173
92500000 59.73546549796138
92600000 96.83073433165605
92700000 91.29275862547274
92800000 94.23371420404438
92900000 94.96057898694461
93000000 97.0418594205235
93100

119100000 66.85904131762125
119200000 58.051975523113946
119300000 90.35013413236511
119400000 89.00136790297138
119500000 48.75192959780447
119600000 64.03972715755567
119700000 82.46478146210467
119800000 100.65709548779573
119900000 53.48348601129492
120000000 84.585028498439
120100000 94.97608492277449
120200000 95.74241168721306
120300000 105.9525201703742
120400000 97.91936785926718
120500000 101.3193268781833
120600000 97.74532344366044
120700000 102.31150152065594
120800000 102.23526032168098
120900000 95.19859286241241
121000000 98.3601320360144
121100000 99.03031162085185
121200000 94.97321595785273
121300000 90.76773293182733
121400000 92.0818277999332
121500000 93.02421744439702
121600000 87.40645745328511
121700000 32.869963261032176
121800000 88.57303925684215
121900000 89.6551881090136
122000000 63.3967543760717
122100000 55.47165995258922
122200000 82.3632742887302
122300000 91.76899618213646
122400000 51.20980996360486
122500000 89.61772748517441
122600000 87.768245216

...

952600000 1.4648738762954323
952700000 1.4898538396521568
952800000 0.6210074858220677
952900000 1.2922265591343045
953000000 1.6397132796283715
953100000 1.7599868734292308
953200000 1.5647383866369367
953300000 1.594500033643367
953400000 0.7711891186037184
953500000 1.1672031849836184
953600000 1.7902738882863833
953700000 1.7134097429853261
953800000 1.6196030571279625
953900000 1.5821842234054584
954000000 1.1447162597580929
954100000 0.6251449539608458
954200000 1.7113186773927946
954300000 1.6937529968727707
954400000 1.7393079992516822
954500000 1.597508073975583
954600000 1.4432068975627443
954700000 0.5258518466097839
954800000 1.4402623212778225
954900000 1.6754320985676567
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.8514956777047384
955100000 1.7589010108579985
955200000 1.8660112772705078
955300000 1.3108749022027981
955400000 1.2520369647045864
955500000 1.3716666206602475
955600000 1.247852111830889
955700000 1.6110948443977586
95580

979000000 1.3672668377342299
best so far: 0
type: [1, 2, 4, 17, 1] 136
cases of this type: 157216
979100000 0.523276738697605
best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.32448950879527927
979300000 0.35884543824823434
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.3254124701577935
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.31424795317055587
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.858709696116689
979700000 1.922053006637622
979800000 1.832743916417756
979900000 1.4375381899455641
980000000 1.4815022701735203
980100000 1.7563102477682786
980200000 1.5525284856542554
980300000 1.4939021610298562
980400000 1.3494714527084886
980500000 1.5124346

998000000 1.2923209855716327
998100000 0.9143916111709594
998200000 1.5101995674358686
998300000 1.3644830299275625
998400000 0.8594701362154366
998500000 1.2456640891035518
998600000 1.3949347549508921
998700000 0.9590380760208507
998800000 1.2068406221821535
998900000 1.3856027487909475
999000000 0.9306990349314471
999100000 1.1502119328081568
999200000 1.3998972971485957
999300000 1.0622155729088543
999400000 1.053152366328535
999500000 1.3709590289724776
999600000 1.1241007252743265
999700000 0.8653020272689832
999800000 1.3462796168964914
999900000 1.3440827643818496
1000000000 0.8277461631344326
1000100000 1.4004407971090773
1000200000 1.4114331004177632
1000300000 0.9394126209623819
1000400000 1.4346011888178474
1000500000 0.2478078580424686
1000600000 0.3277501283885579
1000700000 0.246352350450833
1000800000 0.24556294897391204
1000900000 0.24542956641236013
1001000000 0.2446899597571896
1001100000 0.24430332488500472
1001200000 0.24328868246139354
1001300000 0.242675280498702

1025000000 0.3807808420927703
1025100000 0.24158193972438774
1025200000 0.30474766437808737
1025300000 0.37783074665462774
1025400000 0.25489584097045664
1025500000 0.2820336443166534
1025600000 0.36813774797152743
1025700000 0.2776109460479836
1025800000 0.2480987013627622
1025900000 0.34915500233641134
1026000000 0.3062796396851698
1026100000 0.2124056679886288
1026200000 0.35563271518207656
1026300000 0.31281104626529815
1026400000 0.16824784175241853
1026500000 0.3546888905023005
1026600000 0.32103766833655634
1026700000 0.1749868006778995
1026800000 0.3519320400525735
1026900000 0.3134257419239521
1027000000 0.20381114848966136
1027100000 0.32377880783893137
1027200000 0.3253356659322381
1027300000 0.2282091539149993
1027400000 0.29313214889732664
1027500000 0.33362649269724287
1027600000 0.25059070758060015
1027700000 0.2617141277076814
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.34189485215969684
1027900000 0.2640392337010311
1028000000 0.2

best so far: 0
type: [2, 34, 1, 2, 1] 136
cases of this type: 272
best so far: 0
type: [2, 34, 2, 1, 1] 136
cases of this type: 136
best so far: 0
type: [2, 68, 1, 1, 1] 136
cases of this type: 68
best so far: 0
type: [4, 1, 1, 1, 34] 136
cases of this type: 1336336
1045400000 0.04508085693267849
1045500000 0.052463070750735204
1045600000 0.0467742932333284
1045700000 0.04180569273483422
1045800000 0.04650804321805438
1045900000 0.036202223642702897
1046000000 0.028804090076359118
1046100000 0.010396262699890135
1046200000 0.030341378556383983
1046300000 0.03023164702477455
1046400000 0.033669727472259596
1046500000 0.028627558961395422
1046600000 0.027057772168435655
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.028398243420662484
1046800000 0.014805319649610254
1046900000 0.013876767398620975
1047000000 0.010824169662892819
1047100000 0.005246986681991816
1047200000 0.008651647797470623
1047300000 0.009370676322553555
best so far: 0
type: [4, 1, 1,